In [30]:
import requests, re, pandas as pd, numpy as np, os
from itertools import product
from bs4 import BeautifulSoup as bs
from scipy import stats
import pandas as pd
import numpy as np
from datetime import datetime
from xgboost import XGBClassifier, XGBRegressor
import utils

In [11]:
# ACCESS_TOKEN = 'EAAEau61SMogBAO4Lp0pJaYlJzc67wTychUcwF6gdpLWlet5TqpwnB6k6u0v8DuVAWFaMGO09CQlA7GoOjJnJzTdwzTCO2ZBtCQO721V5U1Qjy5vb7pZCEBC2N5AZAbzVWVYylg1bZCSehy28TTJP7QntMgpxGhD5einLmDxUz3pFfYHPmi47'
ACCESS_TOKEN = 'EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK'

In [12]:
# fields available to read as defined by documentation
FIELDS = ['id','ad_creation_time','ad_delivery_start_time','ad_delivery_stop_time',
          'ad_snapshot_url','ad_creative_bodies','ad_creative_link_captions','ad_creative_link_titles','ad_creative_link_descriptions',
          'languages',
          'page_id','page_name','bylines','currency','spend','impressions','estimated_audience_size','publisher_platforms',
          'demographic_distribution','delivery_by_region']

In [3]:
page_id_list_us = pd.read_csv('Y:/Internship/SoMin/Notebook/report/US-90-days.csv')
page_id_list_us.columns = ['page_id', 'page_name', 'disclaimer', 'amount_spent', 'number_of_ads']

page_id_list_br = pd.read_csv('Y:/Internship/SoMin/Notebook/report/BR-90-days.csv')
page_id_list_br.columns = ['page_id', 'page_name', 'disclaimer', 'amount_spent', 'number_of_ads']

page_id_list_gb = pd.read_csv('Y:/Internship/SoMin/Notebook/report/GB-90-days.csv')
page_id_list_gb.columns = ['page_id', 'page_name', 'disclaimer', 'amount_spent', 'number_of_ads']

In [29]:
page_id_list_us

,page_id,page_name,disclaimer,amount_spent,number_of_ads
0,342294162453301,Planned Parenthood,Planned Parenthood Federation of America,1585113,1594
1,101828156534504,He Gets Us,He Gets Us LLC,983154,765
2,102558898865449,Don't Break What Works,COMPUTER & COMMUNICATIONS INDUSTRY ASSOCIATION,894552,318
3,87186661244,Johns Hopkins Bloomberg School of Public Health,Johns Hopkins Bloomberg School of Public Health,813164,159
4,134498556891640,Val Demings,Val Demings for U.S. Senate,803990,466
...,...,...,...,...,...
17750,107274941522892,East Feliciana Parish Local News,These ads ran without a disclaimer,≤100,1
17751,102639252017167,Warren County Local News,These ads ran without a disclaimer,≤100,1
17752,163295937031815,Linh Nga,These ads ran without a disclaimer,≤100,1
17753,107185384889782,Winn Parish Local News,These ads ran without a disclaimer,≤100,1


In [5]:
page_id_list_br

,page_id,page_name,disclaimer,amount_spent,number_of_ads
0,301774903545521,Brasil Paralelo,Brasil Paralelo,2134607,7510
1,250792278286894,Simone Tebet,MOVIMENTO DEMOCRÁTICO BRASILEIRO,491596,62
2,106558185227603,BP Select,Brasil Paralelo Entretenimento e Educação S/A,404418,1032
3,159103797542,Greenpeace Brasil,Greenpeace Brasil,246796,326
4,411754008869486,Aécio Neves,Aécio Neves,184352,62
...,...,...,...,...,...
11074,110904727326591,Search engine,These ads ran without a disclaimer,≤100,1
11075,210181875766647,Mums in Science,These ads ran without a disclaimer,≤100,1
11076,536483479771472,Danielzinho,Daniel Silva De Lima,≤100,1
11077,163050077800523,Ricardo Rangel,These ads ran without a disclaimer,≤100,1


In [6]:
page_id_list_gb

,page_id,page_name,disclaimer,amount_spent,number_of_ads
0,311627208673,WWF UK,WWF UK,586511,978
1,26698813123,British Red Cross,British Red Cross,471366,174
2,7715002593,UNICEF UK,Unicef UK,441495,124
3,408582579294175,UK government,UK Government,415960,153
4,9250349228,Greenpeace UK,Greenpeace,327378,1149
...,...,...,...,...,...
7453,111329294516219,United Kingdom News - NewsBreak,These ads ran without a disclaimer,≤100,1
7454,233022620980830,Ebrahim Sheik,These ads ran without a disclaimer,≤100,1
7455,104124844567177,The Urban Foxes,These ads ran without a disclaimer,≤100,1
7456,110018661651101,Leadership Focus International Initiative,These ads ran without a disclaimer,≤100,1


In [19]:
# ACCESS_TOKEN = 'EAAEau61SMogBAO4Lp0pJaYlJzc67wTychUcwF6gdpLWlet5TqpwnB6k6u0v8DuVAWFaMGO09CQlA7GoOjJnJzTdwzTCO2ZBtCQO721V5U1Qjy5vb7pZCEBC2N5AZAbzVWVYylg1bZCSehy28TTJP7QntMgpxGhD5einLmDxUz3pFfYHPmi47'
ACCESS_TOKEN = 'EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK'
def request(method='GET', full_url='', resource='', q={}):
    """
    resource: Endpoint on FB Graph API
    q: Access Token, Parameters and Fields according to FB Graph API documentation for specified endpoint
    """
    response = None
    if method == 'GET':
        if full_url:
            response = requests.get(full_url)
        if resource:
            response = requests.get(f'https://graph.facebook.com/v13.0/{resource}', q)
        if response.status_code==400:
            print(f'Error Status Code 400 for URL: {response.url}')
            return response
    return response.json()


def read_endpoint(id_, endpoint, access_token, fields=[], params={}, n=1000000):
    """
    read any endpoint for Facebook Graph API
    
    id_: identifier for any of Business, Account, Campaign, Ad Set, Ad, Creative, Ad Image, Ad Library
    endpoint: endpoint for any of the above according to Facebook Graph API
    access_token: required for usage of Facebook Graph API
    fields: fields to be read for the specified endpoint
    params: parameters to be passed for the specified endpoint
    n: number of desired results to be returned
    """
    q = dict({'access_token': access_token, 'fields': ','.join(fields)}, **params)
    if id_ == '':
        response = request(method='GET', resource=endpoint, q=q)
    else:
        response = request(method='GET', resource=f'{id_}/{endpoint}', q=q)
    if endpoint == '':
        return response
    
    data = []
    while True:
        try:
            data.extend(response['data'])
            if 'next' not in response['paging'].keys():
                break
            if len(data) >= n:
                break
            response = request(method='GET', full_url=response['paging']['next'])
        except Exception as e:
            break # not accessible
    return data


# fields available to read as defined by documentation
FIELDS = ['id','ad_creation_time','ad_delivery_start_time','ad_delivery_stop_time',
          'ad_snapshot_url','ad_creative_bodies','ad_creative_link_captions','ad_creative_link_titles','ad_creative_link_descriptions',
          'languages',
          'page_id','page_name','bylines','currency','spend','impressions','estimated_audience_size','publisher_platforms',
          'demographic_distribution','delivery_by_region']

def obtain_ads_page_name(search_term, country, fields, access_token):
    """
    a function to obtain ads from Ad Library based on given 'search_page_ids' and 'country'
    """
    params = {'search_terms': search_term,
              'search_type': 'KEYWORD_UNORDERED',
              'ad_active_status': 'ALL',
              'ad_reached_countries': [country]}
    
    ads = read_endpoint('', 'ads_archive', access_token, fields, params, 30000)
    
    for ad in ads: # add additional information
#         ad['search_page_ids'] = page_id 
        ad['topic'] = search_term 
        ad['country'] = country
    return ads

def obtain_ads_page_id(page_id, country, fields, access_token):
    """
    a function to obtain ads from Ad Library based on given 'search_page_ids' and 'country'
    """
    params = {'search_page_ids': page_id,
              'media_type': 'IMAGE',
              'ad_active_status': 'ALL',
              'ad_reached_countries': [country]}
    
    ads = read_endpoint('', 'ads_archive', access_token, fields, params, 30000)
    
    for ad in ads: # add additional information
        ad['search_page_ids'] = page_id 
        ad['country'] = country
    return ads

In [28]:
ads = obtain_ads_page_id('26698813123', 'GB', FIELDS, ACCESS_TOKEN)
print(len(ads))

802


In [26]:
page_id_us = page_id_list_us['page_id']
page_id_us = np.unique(page_id_us)[1:]

page_id_br = page_id_list_br['page_id']
page_id_br = np.unique(page_id_br)[1:]

page_id_gb = page_id_list_gb['page_id']
page_id_gb = np.unique(page_id_gb)[1:]

In [27]:
print('US:', len(page_id_us))
print('BR:', len(page_id_br))
print('GB:', len(page_id_gb))

US: 16950
BR: 9726
GB: 6853


In [ ]:
page_id_us

array([       5435784683,        5633824547,        5634829153, ...,
       10150127026615249, 10150132207935252, 10150157803280161],
      dtype=int64)

In [14]:
import os
import pickle
from datetime import datetime
def ensure_folder_exists(folder_path):
    """
    Creates the specified folder if it does not already exist
    """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)


def load_pickle(file_path):
    """
    Loads a pickle from the file system
    """
    return pickle.load(open(file_path, 'rb')) if os.path.isfile(file_path) else None


def save_pickle(file_path, data):
    """
    Saves the specified object as a pickle to the file system
    """
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

raw_dir = f'Y:/Internship/SoMin/Notebook/raw/{str(datetime.today().date()).replace("-","")}'
if not os.path.exists(raw_dir):
    os.makedirs(raw_dir)

# Great Britain

In [ ]:
from itertools import product
# countries = ['US', 'GB', 'BR'] # all available countries for now
# themes = ['ad-tech','ed-tech','agri-tech','fin-tech','bio-tech','health-tech','electric-vehicles','clean-energy',
#           'cloud-computing','quantum-computing','artificial-intelligence','machine-learning','digital-assets',
#           'virtual-reality','cyber-security','software','social-trends','analytics']

all_ads = []
i = 0
for ct in product(page_id_gb, ['GB']):
    if i%100 == 0:
        print(f'Saving ..... batch {i//100}, current id: {i}')
        save_pickle(f'{raw_dir}/adlibrary_ads_gb_90_days_{i//1000}.pkl', all_ads)
        if i%1000 == 0:
            all_ads = []
    print(f'Retrieving Ads for: {ct}, number {i}')
    try:
        ads = obtain_ads_page_id(ct[0], ct[1], FIELDS, ACCESS_TOKEN)
        print('\t', len(ads), 'ads')
        all_ads.extend(ads)
        i+=1
    except Exception as e:
        print('Some error happened:', e)
        
save_pickle(f'{raw_dir}/adlibrary_ads_gb_90_days_{i//1000}.pkl', all_ads)
print(f'Retrieved {len(all_ads)} ads')

Saving ..... batch 0, current id: 0
Retrieving Ads for: (5647744585, 'GB'), number 0
	 0 ads
Retrieving Ads for: (5662929773, 'GB'), number 1
	 11 ads
Retrieving Ads for: (5740844431, 'GB'), number 2
	 3 ads
Retrieving Ads for: (5883973269, 'GB'), number 3
	 786 ads
Retrieving Ads for: (5938132369, 'GB'), number 4
	 6 ads
Retrieving Ads for: (6040534532, 'GB'), number 5
	 0 ads
Retrieving Ads for: (6080640749, 'GB'), number 6
	 0 ads
Retrieving Ads for: (6115848166, 'GB'), number 7
	 3 ads
Retrieving Ads for: (6149699161, 'GB'), number 8
	 0 ads
Retrieving Ads for: (6162236025, 'GB'), number 9
	 2 ads
Retrieving Ads for: (6233536938, 'GB'), number 10
	 251 ads
Retrieving Ads for: (6321018343, 'GB'), number 11
	 1 ads
Retrieving Ads for: (6366026879, 'GB'), number 12
	 169 ads
Retrieving Ads for: (6406164817, 'GB'), number 13
	 3 ads
Retrieving Ads for: (6447117653, 'GB'), number 14
	 1267 ads
Retrieving Ads for: (6574632957, 'GB'), number 15
	 8 ads
Retrieving Ads for: (6587671199, 'GB

	 5 ads
Retrieving Ads for: (25919404073, 'GB'), number 137
	 17 ads
Retrieving Ads for: (26416930992, 'GB'), number 138
	 22 ads
Retrieving Ads for: (26698813123, 'GB'), number 139
	 802 ads
Retrieving Ads for: (26806660922, 'GB'), number 140
	 12 ads
Retrieving Ads for: (27560776046, 'GB'), number 141
	 0 ads
Retrieving Ads for: (27678194634, 'GB'), number 142
	 2 ads
Retrieving Ads for: (27770740767, 'GB'), number 143
	 0 ads
Retrieving Ads for: (28748279914, 'GB'), number 144
	 0 ads
Retrieving Ads for: (28750720737, 'GB'), number 145
	 5 ads
Retrieving Ads for: (28928259418, 'GB'), number 146
	 0 ads
Retrieving Ads for: (29261542197, 'GB'), number 147
	 12 ads
Retrieving Ads for: (29952518646, 'GB'), number 148
	 0 ads
Retrieving Ads for: (30028422818, 'GB'), number 149
	 0 ads
Retrieving Ads for: (30100671591, 'GB'), number 150
	 0 ads
Retrieving Ads for: (30394579038, 'GB'), number 151
	 0 ads
Retrieving Ads for: (30601448467, 'GB'), number 152
	 0 ads
Retrieving Ads for: (30651

	 0 ads
Retrieving Ads for: (78066290984, 'GB'), number 273
	 0 ads
Retrieving Ads for: (78436661801, 'GB'), number 274
	 0 ads
Retrieving Ads for: (78631091926, 'GB'), number 275
	 1 ads
Retrieving Ads for: (79511407410, 'GB'), number 276
	 1 ads
Retrieving Ads for: (80791482166, 'GB'), number 277
	 0 ads
Retrieving Ads for: (81220709615, 'GB'), number 278
	 7 ads
Retrieving Ads for: (81659621206, 'GB'), number 279
	 2 ads
Retrieving Ads for: (81751285098, 'GB'), number 280
	 2 ads
Retrieving Ads for: (83181195937, 'GB'), number 281
	 0 ads
Retrieving Ads for: (84816113788, 'GB'), number 282
	 0 ads
Retrieving Ads for: (85412112798, 'GB'), number 283
	 0 ads
Retrieving Ads for: (85549822738, 'GB'), number 284
	 67 ads
Retrieving Ads for: (85800829429, 'GB'), number 285
	 2 ads
Retrieving Ads for: (86532198605, 'GB'), number 286
	 0 ads
Retrieving Ads for: (86760241759, 'GB'), number 287
	 0 ads
Retrieving Ads for: (87772838729, 'GB'), number 288
	 0 ads
Retrieving Ads for: (8799356418

	 2 ads
Retrieving Ads for: (142972967665, 'GB'), number 407
	 0 ads
Retrieving Ads for: (143140920600, 'GB'), number 408
	 0 ads
Retrieving Ads for: (143666524748, 'GB'), number 409
	 70 ads
Retrieving Ads for: (143957883722, 'GB'), number 410
	 0 ads
Retrieving Ads for: (145666450461, 'GB'), number 411
	 3 ads
Retrieving Ads for: (146599948077, 'GB'), number 412
	 0 ads
Retrieving Ads for: (146998042522, 'GB'), number 413
	 0 ads
Retrieving Ads for: (148617879290, 'GB'), number 414
	 0 ads
Retrieving Ads for: (150632371526, 'GB'), number 415
	 0 ads
Retrieving Ads for: (152981462200, 'GB'), number 416
	 0 ads
Retrieving Ads for: (153494835802, 'GB'), number 417
	 3 ads
Retrieving Ads for: (153660100664, 'GB'), number 418
	 0 ads
Retrieving Ads for: (154312505920, 'GB'), number 419
	 0 ads
Retrieving Ads for: (156180324417, 'GB'), number 420
	 0 ads
Retrieving Ads for: (156259257856, 'GB'), number 421
	 0 ads
Retrieving Ads for: (156409912584, 'GB'), number 422
	 0 ads
Retrieving Ads 

	 0 ads
Retrieving Ads for: (326112022415, 'GB'), number 541
	 0 ads
Retrieving Ads for: (326605791719, 'GB'), number 542
	 0 ads
Retrieving Ads for: (329935353925, 'GB'), number 543
	 0 ads
Retrieving Ads for: (330351984041, 'GB'), number 544
	 0 ads
Retrieving Ads for: (331351247229, 'GB'), number 545
	 2 ads
Retrieving Ads for: (332342972847, 'GB'), number 546
	 12 ads
Retrieving Ads for: (334441960730, 'GB'), number 547
	 3 ads
Retrieving Ads for: (339155437887, 'GB'), number 548
	 1 ads
Retrieving Ads for: (342002108699, 'GB'), number 549
	 14 ads
Retrieving Ads for: (342382261712, 'GB'), number 550
	 6 ads
Retrieving Ads for: (343254621052, 'GB'), number 551
	 1 ads
Retrieving Ads for: (343543623867, 'GB'), number 552
	 4 ads
Retrieving Ads for: (345862640721, 'GB'), number 553
	 0 ads
Retrieving Ads for: (346126507178, 'GB'), number 554
	 2 ads
Retrieving Ads for: (347265340270, 'GB'), number 555
	 2 ads
Retrieving Ads for: (349596960793, 'GB'), number 556
	 9 ads
Retrieving Ads

...

	 0 ads
Retrieving Ads for: (1463126643980071, 'GB'), number 6328
	 12 ads
Retrieving Ads for: (1464057183864929, 'GB'), number 6329
	 1 ads
Retrieving Ads for: (1464101100526640, 'GB'), number 6330
	 0 ads
Retrieving Ads for: (1464779663812914, 'GB'), number 6331
	 1 ads
Retrieving Ads for: (1465121530185867, 'GB'), number 6332
	 0 ads
Retrieving Ads for: (1465968600347274, 'GB'), number 6333
	 0 ads
Retrieving Ads for: (1466420003575396, 'GB'), number 6334
	 0 ads
Retrieving Ads for: (1466956066881119, 'GB'), number 6335
	 0 ads
Retrieving Ads for: (1471272676250535, 'GB'), number 6336
	 0 ads
Retrieving Ads for: (1473097629654129, 'GB'), number 6337
	 0 ads
Retrieving Ads for: (1473321922884514, 'GB'), number 6338
	 0 ads
Retrieving Ads for: (1473388192890892, 'GB'), number 6339
	 0 ads
Retrieving Ads for: (1473735786052133, 'GB'), number 6340
	 0 ads
Retrieving Ads for: (1473803512759360, 'GB'), number 6341
	 0 ads
Retrieving Ads for: (1477945425811579, 'GB'), number 6342
	 0 ads
R

	 24 ads
Retrieving Ads for: (1574840629433114, 'GB'), number 6452
	 15 ads
Retrieving Ads for: (1583568088446259, 'GB'), number 6453
	 0 ads
Retrieving Ads for: (1583635131917873, 'GB'), number 6454
	 0 ads
Retrieving Ads for: (1586251881644270, 'GB'), number 6455
	 125 ads
Retrieving Ads for: (1587504104861466, 'GB'), number 6456
	 0 ads
Retrieving Ads for: (1587544891571761, 'GB'), number 6457
	 0 ads
Retrieving Ads for: (1589251641323448, 'GB'), number 6458
	 2 ads
Retrieving Ads for: (1589386114454472, 'GB'), number 6459
	 0 ads
Retrieving Ads for: (1589464524604653, 'GB'), number 6460
	 1 ads
Retrieving Ads for: (1590952364500354, 'GB'), number 6461
	 2 ads
Retrieving Ads for: (1591011817813461, 'GB'), number 6462
	 0 ads
Retrieving Ads for: (1591520794437621, 'GB'), number 6463
	 0 ads
Retrieving Ads for: (1591849810839431, 'GB'), number 6464
	 3 ads
Retrieving Ads for: (1591920221104299, 'GB'), number 6465
	 6 ads
Retrieving Ads for: (1593820680884174, 'GB'), number 6466
	 2 ad

	 0 ads
Retrieving Ads for: (1714579138857138, 'GB'), number 6576
	 0 ads
Retrieving Ads for: (1715537042054967, 'GB'), number 6577
	 0 ads
Retrieving Ads for: (1718863814992513, 'GB'), number 6578
	 2 ads
Retrieving Ads for: (1718960811700485, 'GB'), number 6579
	 0 ads
Retrieving Ads for: (1720702718165314, 'GB'), number 6580
	 0 ads
Retrieving Ads for: (1721275931466984, 'GB'), number 6581
	 8 ads
Retrieving Ads for: (1724526554225273, 'GB'), number 6582
	 0 ads
Retrieving Ads for: (1725037007557026, 'GB'), number 6583
	 0 ads
Retrieving Ads for: (1727970884168768, 'GB'), number 6584
	 1 ads
Retrieving Ads for: (1731328363835875, 'GB'), number 6585
	 0 ads
Retrieving Ads for: (1738524969810037, 'GB'), number 6586
	 255 ads
Retrieving Ads for: (1739923619633397, 'GB'), number 6587
	 0 ads
Retrieving Ads for: (1741177002770228, 'GB'), number 6588
	 0 ads
Retrieving Ads for: (1744895835728090, 'GB'), number 6589
	 1 ads
Retrieving Ads for: (1746907342219464, 'GB'), number 6590
	 0 ads


	 150 ads
Saving ..... batch 67, current id: 6700
Retrieving Ads for: (1923608401265180, 'GB'), number 6700
	 0 ads
Retrieving Ads for: (1928866947332662, 'GB'), number 6701
	 0 ads
Retrieving Ads for: (1929320457384716, 'GB'), number 6702
	 3 ads
Retrieving Ads for: (1929801517326480, 'GB'), number 6703
	 0 ads
Retrieving Ads for: (1929915730481059, 'GB'), number 6704
	 2 ads
Retrieving Ads for: (1930410900543324, 'GB'), number 6705
	 0 ads
Retrieving Ads for: (1930888720475126, 'GB'), number 6706
	 2 ads
Retrieving Ads for: (1931209670264436, 'GB'), number 6707
	 0 ads
Retrieving Ads for: (1933283223404242, 'GB'), number 6708
	 0 ads
Retrieving Ads for: (1934566156780439, 'GB'), number 6709
	 78 ads
Retrieving Ads for: (1935310150039577, 'GB'), number 6710
	 0 ads
Retrieving Ads for: (1935868476637300, 'GB'), number 6711
	 4 ads
Retrieving Ads for: (1937193049741596, 'GB'), number 6712
	 1 ads
Retrieving Ads for: (1937547913206353, 'GB'), number 6713
	 0 ads
Retrieving Ads for: (1938

	 12 ads
Retrieving Ads for: (2296555250567080, 'GB'), number 6823
	 0 ads
Retrieving Ads for: (2296667497229041, 'GB'), number 6824
	 1 ads
Retrieving Ads for: (2297926640486522, 'GB'), number 6825
	 363 ads
Retrieving Ads for: (2303250593259491, 'GB'), number 6826
	 26 ads
Retrieving Ads for: (2307838149231785, 'GB'), number 6827
	 2 ads
Retrieving Ads for: (2328990920656003, 'GB'), number 6828
	 0 ads
Retrieving Ads for: (2347565375570069, 'GB'), number 6829
	 0 ads
Retrieving Ads for: (2375299949384184, 'GB'), number 6830
	 3 ads
Retrieving Ads for: (2378768448836033, 'GB'), number 6831
	 0 ads
Retrieving Ads for: (2382280078671812, 'GB'), number 6832
	 0 ads
Retrieving Ads for: (2389668497727588, 'GB'), number 6833
	 25 ads
Retrieving Ads for: (2422768761084748, 'GB'), number 6834
	 0 ads
Retrieving Ads for: (2435680456708504, 'GB'), number 6835
	 0 ads
Retrieving Ads for: (2443299312347521, 'GB'), number 6836
	 72 ads
Retrieving Ads for: (2449574395105133, 'GB'), number 6837
	 3 

# Brazil

In [ ]:
from itertools import product
# countries = ['US', 'GB', 'BR'] # all available countries for now
# themes = ['ad-tech','ed-tech','agri-tech','fin-tech','bio-tech','health-tech','electric-vehicles','clean-energy',
#           'cloud-computing','quantum-computing','artificial-intelligence','machine-learning','digital-assets',
#           'virtual-reality','cyber-security','software','social-trends','analytics']

all_ads = []
i = 0
total_ads = 0
for ct in product(page_id_br, ['BR']):
    if i%100 == 0:
        print(f'Saving ..... batch {i//100}, current id: {i}')
        save_pickle(f'{raw_dir}/adlibrary_ads_br_90_days_{i//1000}.pkl', all_ads)
        if i%1000 == 0:
            all_ads = []
    print(f'Retrieving Ads for: {ct}, number {i}')
    try:
        ads = obtain_ads_page_id(ct[0], ct[1], FIELDS, ACCESS_TOKEN)
        total_ads += len(ads)
        print('\t', len(ads), 'ads, total', total_ads)
        all_ads.extend(ads)
        i+=1
    except Exception as e:
        print('Some error happened:', e)
        
save_pickle(f'{raw_dir}/adlibrary_ads_br_90_days_{i//1000}.pkl', all_ads)
print(f'Retrieved {len(all_ads)} ads')

Saving ..... batch 0, current id: 0
Retrieving Ads for: (5647744585, 'BR'), number 0
	 3 ads, total 3
Retrieving Ads for: (6085906515, 'BR'), number 1
	 0 ads, total 3
Retrieving Ads for: (7619396355, 'BR'), number 2
	 0 ads, total 3
Retrieving Ads for: (7783116506, 'BR'), number 3
	 0 ads, total 3
Retrieving Ads for: (8539814909, 'BR'), number 4
	 0 ads, total 3
Retrieving Ads for: (12185972707, 'BR'), number 5
	 0 ads, total 3
Retrieving Ads for: (17043549797, 'BR'), number 6
	 4 ads, total 7
Retrieving Ads for: (22760563564, 'BR'), number 7
	 2 ads, total 9
Retrieving Ads for: (24287259392, 'BR'), number 8
	 0 ads, total 9
Retrieving Ads for: (26897053321, 'BR'), number 9
	 0 ads, total 9
Retrieving Ads for: (27678194634, 'BR'), number 10
	 2 ads, total 11
Retrieving Ads for: (31292782350, 'BR'), number 11
	 26 ads, total 37
Retrieving Ads for: (36085812354, 'BR'), number 12
	 2 ads, total 39
Retrieving Ads for: (39644961958, 'BR'), number 13
	 0 ads, total 39
Retrieving Ads for: (3

Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK&fields=id%2Cad_creation_time%2Cad_delivery_start_time%2Cad_delivery_stop_time%2Cad_snapshot_url%2Cad_creative_bodies%2Cad_creative_link_captions%2Cad_creative_link_titles%2Cad_creative_link_descriptions%2Clanguages%2Cpage_id%2Cpage_name%2Cbylines%2Ccurrency%2Cspend%2Cimpressions%2Cestimated_audience_size%2Cpublisher_platforms%2Cdemographic_distribution%2Cdelivery_by_region&search_page_ids=100076366921990&media_type=IMAGE&ad_active_status=ALL&ad_reached_countries=BR
	 0 ads, total 623
Retrieving Ads for: (100100305866299, 'BR'), number 106
	 0 ads, total 623
Retrieving Ads for: (100103949013499, 'BR'), number 107
	 0 ads, total 623
Retrieving Ads for: (100106159009340, 'BR'), number 108
	 3 ads, total 626
Retrie

	 0 ads, total 822
Retrieving Ads for: (100281858452239, 'BR'), number 205
	 0 ads, total 822
Retrieving Ads for: (100282012031785, 'BR'), number 206
	 0 ads, total 822
Retrieving Ads for: (100283312645780, 'BR'), number 207
	 0 ads, total 822
Retrieving Ads for: (100285212597283, 'BR'), number 208
	 0 ads, total 822
Retrieving Ads for: (100293621704902, 'BR'), number 209
	 0 ads, total 822
Retrieving Ads for: (100294668152772, 'BR'), number 210
	 0 ads, total 822
Retrieving Ads for: (100295402695450, 'BR'), number 211
	 1 ads, total 823
Retrieving Ads for: (100296319269692, 'BR'), number 212
	 1 ads, total 824
Retrieving Ads for: (100300555155590, 'BR'), number 213
	 0 ads, total 824
Retrieving Ads for: (100306552231937, 'BR'), number 214
	 1 ads, total 825
Retrieving Ads for: (100308005591904, 'BR'), number 215
	 0 ads, total 825
Retrieving Ads for: (100308035003468, 'BR'), number 216
	 0 ads, total 825
Retrieving Ads for: (100308622165800, 'BR'), number 217
	 0 ads, total 825
Retrie

	 0 ads, total 1015
Retrieving Ads for: (100480825586733, 'BR'), number 314
	 0 ads, total 1015
Retrieving Ads for: (100481471619855, 'BR'), number 315
	 0 ads, total 1015
Retrieving Ads for: (100484845123222, 'BR'), number 316
	 0 ads, total 1015
Retrieving Ads for: (100485415109106, 'BR'), number 317
	 0 ads, total 1015
Retrieving Ads for: (100485928025444, 'BR'), number 318
	 0 ads, total 1015
Retrieving Ads for: (100487282049483, 'BR'), number 319
	 0 ads, total 1015
Retrieving Ads for: (100489464695466, 'BR'), number 320
	 0 ads, total 1015
Retrieving Ads for: (100493319325173, 'BR'), number 321
	 0 ads, total 1015
Retrieving Ads for: (100505289289765, 'BR'), number 322
	 0 ads, total 1015
Retrieving Ads for: (100505418999694, 'BR'), number 323
	 0 ads, total 1015
Retrieving Ads for: (100506212659974, 'BR'), number 324
	 0 ads, total 1015
Retrieving Ads for: (100509705346931, 'BR'), number 325
	 0 ads, total 1015
Retrieving Ads for: (100513511837101, 'BR'), number 326
	 0 ads, tot

	 0 ads, total 1083
Retrieving Ads for: (100707409013856, 'BR'), number 422
	 0 ads, total 1083
Retrieving Ads for: (100709958332212, 'BR'), number 423
	 9 ads, total 1092
Retrieving Ads for: (100710412444302, 'BR'), number 424
	 3 ads, total 1095
Retrieving Ads for: (100711292644241, 'BR'), number 425
	 0 ads, total 1095
Retrieving Ads for: (100713212642573, 'BR'), number 426
	 0 ads, total 1095
Retrieving Ads for: (100715679098743, 'BR'), number 427
	 0 ads, total 1095
Retrieving Ads for: (100716292514554, 'BR'), number 428
	 0 ads, total 1095
Retrieving Ads for: (100717759182932, 'BR'), number 429
	 0 ads, total 1095
Retrieving Ads for: (100719282065822, 'BR'), number 430
	 0 ads, total 1095
Retrieving Ads for: (100722409286481, 'BR'), number 431
	 0 ads, total 1095
Retrieving Ads for: (100722782289750, 'BR'), number 432
	 3 ads, total 1098
Retrieving Ads for: (100723398684770, 'BR'), number 433
	 0 ads, total 1098
Retrieving Ads for: (100725552156623, 'BR'), number 434
	 0 ads, tot

...

Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK&fields=id%2Cad_creation_time%2Cad_delivery_start_time%2Cad_delivery_stop_time%2Cad_snapshot_url%2Cad_creative_bodies%2Cad_creative_link_captions%2Cad_creative_link_titles%2Cad_creative_link_descriptions%2Clanguages%2Cpage_id%2Cpage_name%2Cbylines%2Ccurrency%2Cspend%2Cimpressions%2Cestimated_audience_size%2Cpublisher_platforms%2Cdemographic_distribution%2Cdelivery_by_region&search_page_ids=304158673086753&media_type=IMAGE&ad_active_status=ALL&ad_reached_countries=BR
	 0 ads, total 36983
Retrieving Ads for: (304315266384698, 'BR'), number 5753
Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i

Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK&fields=id%2Cad_creation_time%2Cad_delivery_start_time%2Cad_delivery_stop_time%2Cad_snapshot_url%2Cad_creative_bodies%2Cad_creative_link_captions%2Cad_creative_link_titles%2Cad_creative_link_descriptions%2Clanguages%2Cpage_id%2Cpage_name%2Cbylines%2Ccurrency%2Cspend%2Cimpressions%2Cestimated_audience_size%2Cpublisher_platforms%2Cdemographic_distribution%2Cdelivery_by_region&search_page_ids=305663189589587&media_type=IMAGE&ad_active_status=ALL&ad_reached_countries=BR
	 0 ads, total 36983
Retrieving Ads for: (305962019571797, 'BR'), number 5763
Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i

Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK&fields=id%2Cad_creation_time%2Cad_delivery_start_time%2Cad_delivery_stop_time%2Cad_snapshot_url%2Cad_creative_bodies%2Cad_creative_link_captions%2Cad_creative_link_titles%2Cad_creative_link_descriptions%2Clanguages%2Cpage_id%2Cpage_name%2Cbylines%2Ccurrency%2Cspend%2Cimpressions%2Cestimated_audience_size%2Cpublisher_platforms%2Cdemographic_distribution%2Cdelivery_by_region&search_page_ids=306946052788157&media_type=IMAGE&ad_active_status=ALL&ad_reached_countries=BR
	 0 ads, total 36983
Retrieving Ads for: (306954859407257, 'BR'), number 5773
Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i

Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK&fields=id%2Cad_creation_time%2Cad_delivery_start_time%2Cad_delivery_stop_time%2Cad_snapshot_url%2Cad_creative_bodies%2Cad_creative_link_captions%2Cad_creative_link_titles%2Cad_creative_link_descriptions%2Clanguages%2Cpage_id%2Cpage_name%2Cbylines%2Ccurrency%2Cspend%2Cimpressions%2Cestimated_audience_size%2Cpublisher_platforms%2Cdemographic_distribution%2Cdelivery_by_region&search_page_ids=308473539215980&media_type=IMAGE&ad_active_status=ALL&ad_reached_countries=BR
	 0 ads, total 36983
Retrieving Ads for: (308593399238061, 'BR'), number 5783
Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i

KeyboardInterrupt: 

In [ ]:
total_ads
len(all_ads)

8856

In [ ]:
import os
import pickle
from datetime import datetime
def load_pickle(file_path):
    """
    Loads a pickle from the file system
    """
    return pickle.load(open(file_path, 'rb')) if os.path.isfile(file_path) else None
target = 'Y:/Internship/SoMin/Notebook/raw/20220526/adlibrary_ads_br_90_days_7.pkl'
all_ads = load_pickle(target)
len(all_ads)

3082

In [ ]:
total_ads = 0
for i in range(8):
    target = f'Y:/Internship/SoMin/Notebook/raw/20220526/adlibrary_ads_br_90_days_{i}.pkl'
    all_ads = load_pickle(target)
    total_ads += len(all_ads)
    print(total_ads)

2786
4721
6366
10651
22610
32273
36961
40043


In [ ]:
def save_pickle(file_path, data):
    """
    Saves the specified object as a pickle to the file system
    """
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
target = 'Y:/Internship/SoMin/Notebook/raw/20220526/adlibrary_ads_br_90_days_7.pkl'
all_ads = load_pickle(target)
len(all_ads)

3082

In [ ]:
i = 7501
raw_dir = 'Y:/Internship/SoMin/Notebook/raw/20220526'
total_ads = 38588
for ct in product(page_id_br, ['BR']):
    if i%100 == 0:
        print(f'Saving ..... batch {i//100}, current id: {i}')
        save_pickle(f'{raw_dir}/adlibrary_ads_br_90_days_{i//1000}.pkl', all_ads)
        if i%1000 == 0:
            all_ads = []
    print(f'Retrieving Ads for: {ct}, number {i}')
    try:
        ads = obtain_ads_page_id(ct[0], ct[1], FIELDS, ACCESS_TOKEN)
        total_ads += len(ads)
        print('\t', len(ads), 'ads, total', total_ads)
        all_ads.extend(ads)
        i+=1
    except Exception as e:
        print('Some error happened:', e)
        
save_pickle(f'{raw_dir}/adlibrary_ads_br_90_days_{i//1000}.pkl', all_ads)
print(f'Retrieved {len(all_ads)} ads')

Retrieving Ads for: (5647744585, 'BR'), number 7501
	 3 ads, total 38591
Retrieving Ads for: (6085906515, 'BR'), number 7502
	 0 ads, total 38591
Retrieving Ads for: (7619396355, 'BR'), number 7503
	 0 ads, total 38591
Retrieving Ads for: (7783116506, 'BR'), number 7504
	 0 ads, total 38591
Retrieving Ads for: (8539814909, 'BR'), number 7505
	 0 ads, total 38591
Retrieving Ads for: (12185972707, 'BR'), number 7506
	 0 ads, total 38591
Retrieving Ads for: (17043549797, 'BR'), number 7507
	 4 ads, total 38595
Retrieving Ads for: (22760563564, 'BR'), number 7508
	 2 ads, total 38597
Retrieving Ads for: (24287259392, 'BR'), number 7509
	 0 ads, total 38597
Retrieving Ads for: (26897053321, 'BR'), number 7510
	 0 ads, total 38597
Retrieving Ads for: (27678194634, 'BR'), number 7511
	 2 ads, total 38599
Retrieving Ads for: (31292782350, 'BR'), number 7512
	 26 ads, total 38625
Retrieving Ads for: (36085812354, 'BR'), number 7513
	 2 ads, total 38627
Retrieving Ads for: (39644961958, 'BR'), n

Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK&fields=id%2Cad_creation_time%2Cad_delivery_start_time%2Cad_delivery_stop_time%2Cad_snapshot_url%2Cad_creative_bodies%2Cad_creative_link_captions%2Cad_creative_link_titles%2Cad_creative_link_descriptions%2Clanguages%2Cpage_id%2Cpage_name%2Cbylines%2Ccurrency%2Cspend%2Cimpressions%2Cestimated_audience_size%2Cpublisher_platforms%2Cdemographic_distribution%2Cdelivery_by_region&search_page_ids=100076366921990&media_type=IMAGE&ad_active_status=ALL&ad_reached_countries=BR
	 0 ads, total 39212
Retrieving Ads for: (100100305866299, 'BR'), number 7607
	 0 ads, total 39212
Retrieving Ads for: (100103949013499, 'BR'), number 7608
	 0 ads, total 39212
Retrieving Ads for: (100106159009340, 'BR'), number 7609
	 3 ads, total 3

	 0 ads, total 39396
Retrieving Ads for: (100273806001620, 'BR'), number 7702
	 0 ads, total 39396
Retrieving Ads for: (100275083494691, 'BR'), number 7703
	 16 ads, total 39412
Retrieving Ads for: (100275178876108, 'BR'), number 7704
	 0 ads, total 39412
Retrieving Ads for: (100279589321213, 'BR'), number 7705
	 0 ads, total 39412
Retrieving Ads for: (100281858452239, 'BR'), number 7706
	 0 ads, total 39412
Retrieving Ads for: (100282012031785, 'BR'), number 7707
	 0 ads, total 39412
Retrieving Ads for: (100283312645780, 'BR'), number 7708
	 0 ads, total 39412
Retrieving Ads for: (100285212597283, 'BR'), number 7709
	 0 ads, total 39412
Retrieving Ads for: (100293621704902, 'BR'), number 7710
	 0 ads, total 39412
Retrieving Ads for: (100294668152772, 'BR'), number 7711
	 0 ads, total 39412
Retrieving Ads for: (100295402695450, 'BR'), number 7712
	 1 ads, total 39413
Retrieving Ads for: (100296319269692, 'BR'), number 7713
	 1 ads, total 39414
Retrieving Ads for: (100300555155590, 'BR'

	 0 ads, total 39559
Retrieving Ads for: (100470085776947, 'BR'), number 7807
	 0 ads, total 39559
Retrieving Ads for: (100471782499807, 'BR'), number 7808
	 0 ads, total 39559
Retrieving Ads for: (100474221848197, 'BR'), number 7809
	 44 ads, total 39603
Retrieving Ads for: (100475082251155, 'BR'), number 7810
	 0 ads, total 39603
Retrieving Ads for: (100475402294243, 'BR'), number 7811
	 0 ads, total 39603
Retrieving Ads for: (100478009254664, 'BR'), number 7812
	 0 ads, total 39603
Retrieving Ads for: (100478458356433, 'BR'), number 7813
	 2 ads, total 39605
Retrieving Ads for: (100479411618465, 'BR'), number 7814
	 0 ads, total 39605
Retrieving Ads for: (100480825586733, 'BR'), number 7815
	 0 ads, total 39605
Retrieving Ads for: (100481471619855, 'BR'), number 7816
	 0 ads, total 39605
Retrieving Ads for: (100484845123222, 'BR'), number 7817
	 0 ads, total 39605
Retrieving Ads for: (100485415109106, 'BR'), number 7818
	 0 ads, total 39605
Retrieving Ads for: (100485928025444, 'BR'

	 10 ads, total 39672
Retrieving Ads for: (100681498935670, 'BR'), number 7912
	 1 ads, total 39673
Retrieving Ads for: (100682332101237, 'BR'), number 7913
	 0 ads, total 39673
Retrieving Ads for: (100689865413766, 'BR'), number 7914
	 0 ads, total 39673
Retrieving Ads for: (100690552356813, 'BR'), number 7915
	 0 ads, total 39673
Retrieving Ads for: (100692935469985, 'BR'), number 7916
	 0 ads, total 39673
Retrieving Ads for: (100697671985410, 'BR'), number 7917
	 0 ads, total 39673
Retrieving Ads for: (100698881704966, 'BR'), number 7918
	 0 ads, total 39673
Retrieving Ads for: (100700572286065, 'BR'), number 7919
	 0 ads, total 39673
Retrieving Ads for: (100704899231458, 'BR'), number 7920
	 0 ads, total 39673
Retrieving Ads for: (100706058941078, 'BR'), number 7921
	 0 ads, total 39673
Retrieving Ads for: (100706812506241, 'BR'), number 7922
	 0 ads, total 39673
Retrieving Ads for: (100707409013856, 'BR'), number 7923
	 0 ads, total 39673
Retrieving Ads for: (100709958332212, 'BR'

...

Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK&fields=id%2Cad_creation_time%2Cad_delivery_start_time%2Cad_delivery_stop_time%2Cad_snapshot_url%2Cad_creative_bodies%2Cad_creative_link_captions%2Cad_creative_link_titles%2Cad_creative_link_descriptions%2Clanguages%2Cpage_id%2Cpage_name%2Cbylines%2Ccurrency%2Cspend%2Cimpressions%2Cestimated_audience_size%2Cpublisher_platforms%2Cdemographic_distribution%2Cdelivery_by_region&search_page_ids=1868600160099570&media_type=IMAGE&ad_active_status=ALL&ad_reached_countries=BR
	 0 ads, total 108701
Retrieving Ads for: (1868600730097718, 'BR'), number 16883
Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7

Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK&fields=id%2Cad_creation_time%2Cad_delivery_start_time%2Cad_delivery_stop_time%2Cad_snapshot_url%2Cad_creative_bodies%2Cad_creative_link_captions%2Cad_creative_link_titles%2Cad_creative_link_descriptions%2Clanguages%2Cpage_id%2Cpage_name%2Cbylines%2Ccurrency%2Cspend%2Cimpressions%2Cestimated_audience_size%2Cpublisher_platforms%2Cdemographic_distribution%2Cdelivery_by_region&search_page_ids=1876823049308268&media_type=IMAGE&ad_active_status=ALL&ad_reached_countries=BR
	 0 ads, total 108701
Retrieving Ads for: (1877959845813046, 'BR'), number 16893
Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7

Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK&fields=id%2Cad_creation_time%2Cad_delivery_start_time%2Cad_delivery_stop_time%2Cad_snapshot_url%2Cad_creative_bodies%2Cad_creative_link_captions%2Cad_creative_link_titles%2Cad_creative_link_descriptions%2Clanguages%2Cpage_id%2Cpage_name%2Cbylines%2Ccurrency%2Cspend%2Cimpressions%2Cestimated_audience_size%2Cpublisher_platforms%2Cdemographic_distribution%2Cdelivery_by_region&search_page_ids=1886454284985367&media_type=IMAGE&ad_active_status=ALL&ad_reached_countries=BR
	 0 ads, total 108701
Retrieving Ads for: (1886624231551883, 'BR'), number 16903
Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7

Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7s49i4ZBecZAZBnnV86z5HMCZAaYp1s2WoUj5P5PzYXTX6Gao2awkZBBudMZBubxwlG3UeRUZBCsBTb5GCeHbAsJFdf0Bal6Rpou4ORNt56cpVpK&fields=id%2Cad_creation_time%2Cad_delivery_start_time%2Cad_delivery_stop_time%2Cad_snapshot_url%2Cad_creative_bodies%2Cad_creative_link_captions%2Cad_creative_link_titles%2Cad_creative_link_descriptions%2Clanguages%2Cpage_id%2Cpage_name%2Cbylines%2Ccurrency%2Cspend%2Cimpressions%2Cestimated_audience_size%2Cpublisher_platforms%2Cdemographic_distribution%2Cdelivery_by_region&search_page_ids=1894520370802455&media_type=IMAGE&ad_active_status=ALL&ad_reached_countries=BR
	 0 ads, total 108701
Retrieving Ads for: (1896179463730113, 'BR'), number 16913
Error Status Code 400 for URL: https://graph.facebook.com/v13.0/ads_archive?access_token=EAAEau61SMogBAKXf5ZCiYZC2WkmfCAkZCEz1cQoZBTgg4AdxRAAmG5ZAcH8hrSEsXPDZBPFodOdKr7

KeyboardInterrupt: 

# US

In [ ]:
from itertools import product
# countries = ['US', 'GB', 'BR'] # all available countries for now
# themes = ['ad-tech','ed-tech','agri-tech','fin-tech','bio-tech','health-tech','electric-vehicles','clean-energy',
#           'cloud-computing','quantum-computing','artificial-intelligence','machine-learning','digital-assets',
#           'virtual-reality','cyber-security','software','social-trends','analytics']
raw_dir = 'Y:/Internship/SoMin/Notebook/raw/20220526'
all_ads = []
i = 0
total_ads = 0
for ct in product(page_id_us, ['US']):
    if i%100 == 0:
        print(f'Saving ..... batch {i//100}, current id: {i}')
        save_pickle(f'{raw_dir}/adlibrary_ads_us_90_days_{i//1000}.pkl', all_ads)
        if i%1000 == 0:
            all_ads = []
    print(f'Retrieving Ads for: {ct}, number {i}')
    try:
        ads = obtain_ads_page_id(ct[0], ct[1], FIELDS, ACCESS_TOKEN)
        total_ads += len(ads)
        print('\t', len(ads), 'ads, total', total_ads)
        all_ads.extend(ads)
        i+=1
    except Exception as e:
        print('Some error happened:', e)
        
save_pickle(f'{raw_dir}/adlibrary_ads_us_90_days_{i//1000}.pkl', all_ads)
print(f'Retrieved {len(all_ads)} ads')

In [ ]:
import os
import pickle
from datetime import datetime
def ensure_folder_exists(folder_path):
    """
    Creates the specified folder if it does not already exist
    """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)


def load_pickle(file_path):
    """
    Loads a pickle from the file system
    """
    return pickle.load(open(file_path, 'rb')) if os.path.isfile(file_path) else None


def save_pickle(file_path, data):
    """
    Saves the specified object as a pickle to the file system
    """
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

raw_dir = f'Y:/Internship/SoMin/Notebook/raw/{str(datetime.today().date()).replace("-","")}'
if not os.path.exists(raw_dir):
    os.makedirs(raw_dir)

# pickle.dumps(all_ads, f'{raw_dir}/adlibrary_ads.pkl')
save_pickle(f'{raw_dir}/adlibrary_ads_us_90_days.pkl', all_ads)

In [ ]:
all_ads = []
for ct in product(page_id_br, ['BR']):
    print('Retrieving for BR')
    print(f'Retrieving Ads for: {ct}')
    ads = obtain_ads_page_id(ct[0], ct[1], FIELDS, ACCESS_TOKEN)
    print('\t', len(ads), 'ads')
    all_ads.extend(ads)
print(f'Retrieved {len(all_ads)} ads')

In [ ]:
all_ads = []
for ct in product(page_id_gb, ['GB']):
    print('Retrieving for GB')
    print(f'Retrieving Ads for: {ct}')
    ads = obtain_ads_page_id(ct[0], ct[1], FIELDS, ACCESS_TOKEN)
    print('\t', len(ads), 'ads')
    all_ads.extend(ads)
    
print(f'Retrieved {len(all_ads)} ads')

In [ ]:
def get_image_url(source):
    """
    function to extract image_url from the retreived data from Ad Library API
    Why?: the URL obtained via the endpoint is not an image_url but instead a url for a snapshot of the entire ad
    """
    pattern = r'"resized_image_url":"(?:\\.|[^"\\])*"' # find resized_image_url : <image_url>
    img_url = re.findall(pattern, str(source))[0].split('"resized_image_url":')[1].replace('\\', '').replace('"', '').replace("'", '')
    return img_url

def process_ads(ads):
    """
    processes Ads retrieved from Ads Archive (FB Ad Library)
    """
    df = pd.DataFrame(ads)    
    clean = []
    for _, row in df.iterrows():
        m = {}
        m['search_term'] = row['topic']
        m['country'] = row['country']
        m['page_id'] = row['page_id']
        m['page_name'] = row['page_name']
        
        m['ad_id'] = row['id']
        
        m['ad_creation_date'] = row['ad_creation_time']
        if m['ad_creation_date'] is np.nan:
            continue
        m['ad_creation_month'] = int(m['ad_creation_date'][5:7])
        
        
        m['delivery_start'] = row['ad_delivery_start_time']
        if m['delivery_start'] is np.nan:
            continue
            
        m['delivery_stop'] = row['ad_delivery_stop_time']
        if m['delivery_stop'] is np.nan:
            continue
        
        start = datetime.strptime(m['delivery_start'], '%Y-%m-%d')
        stop = datetime.strptime(m['delivery_stop'], '%Y-%m-%d')
        m['delivery_period'] = (stop-start).days
        
        m['ad_url'] = row['ad_snapshot_url']
        
#         try:
#             source = soup(requests.get(row['ad_snapshot_url']).content, 'html.parser')
#             m['image_url'] = get_image_url(source)
#         except:
#             m['image_url'] = np.nan # some ads are wrongly classified as having images
        
        ad_body = row['ad_creative_bodies']
        m['ad_body'] = ad_body[0] if (type(ad_body) == list) else ''
        m['uses_multi_body'] = 1 if ((type(ad_body) == list) and (len(set(ad_body)) > 1)) else 0
        
        link_capt = row['ad_creative_link_captions']
        m['link_caption'] = link_capt[0] if (type(link_capt) == list) else ''
        m['uses_multi_capt'] = 1 if ((type(link_capt) == list) and (len(set(link_capt)) > 1)) else 0
        
        link_title = row['ad_creative_link_titles']
        m['link_title'] = link_title[0] if (type(link_title) == list) else ''
        m['uses_multi_title'] = 1 if ((type(link_title) == list) and (len(set(link_title)) > 1)) else 0
        
        link_desc = row['ad_creative_link_descriptions']
        m['link_description'] = link_desc[0] if (type(link_desc) == list) else ''
        m['uses_multi_desc'] = 1 if ((type(link_desc) == list) and (len(set(link_desc)) > 1)) else 0
        
        lang = row['languages']
        m['language'] = ','.join(lang) if (type(lang) == list) else ''
        m['uses_multi_lang'] = 1 if ((type(lang) == list) and (len(set(lang)) > 1)) else 0
        
        m['funded_by'] = row['bylines'] if type(row['bylines']) == str else ''
        m['currency'] = row['currency']
        
        spend = row['spend']
        m['min_spend'] = spend.get('lower_bound', np.nan)
        m['max_spend'] = spend.get('upper_bound', np.nan)
        
        impressions = row['impressions']
        m['min_impressions'] = impressions.get('lower_bound', np.nan)
        m['max_impressions'] = impressions.get('upper_bound', np.nan)
        
        audience_size = row['estimated_audience_size']
        m['min_audience_size'] = audience_size.get('lower_bound', np.nan) if type(audience_size)==dict else np.nan
        m['max_audience_size'] = audience_size.get('upper_bound', np.nan) if type(audience_size)==dict else np.nan
        
        
        if m['max_impressions'] is np.nan or m['min_impressions'] is np.nan: 
            m['impressions'] = 0
            continue
        else:
            m['impressions'] = (int(m['max_impressions']) + int(m['min_impressions']))/2
            
        if m['max_spend'] is np.nan or m['min_spend'] is np.nan: 
            m['spend'] = 0
            continue
        else:
            m['spend'] = (int(m['max_spend']) + int(m['min_spend']))/2
        
        
        if m['max_impressions'] is np.nan or m['max_impressions'] == 0:
            m['CPM'] = 0
            continue
        else:
            m['CPM'] = int(m['max_spend']) / int(m['max_impressions']) * 1000
        
        
        if m['max_impressions'] is np.nan or m['max_impressions'] == 0:
            m['CPM2'] = 0
            continue
        else:
            m['CPM2'] = int(m['spend']) / int(m['impressions']) * 1000
        
        
        
        platforms = row['publisher_platforms']
        m['facebook'] = 1 if ((type(platforms) == list) and ('facebook' in platforms)) else 0
        m['instagram'] = 1 if ((type(platforms) == list) and ('instagram' in platforms)) else 0
        m['messenger'] = 1 if ((type(platforms) == list) and ('messenger' in platforms)) else 0
        m['audience_network'] = 1 if ((type(platforms) == list) and ('audience_network' in platforms)) else 0
        
        m['demographic_dist'] = row['demographic_distribution']
        clean.append(m)
        
    return pd.DataFrame(clean)